## General idea

- Objects are in a fixed hierarchy
- We cannot sync an object without its dependencies without getting a broken state
- So: When resolving diffs between high-low side, we need to resolve the whole tree of dependencies at the same time

In [1]:
import syft as sy
from syft.types.syft_object import SyftObject
from syft import ActionObject
from syft.service.user.user import UserCreate
from syft.node.credentials import SyftVerifyKey
from syft.service.action.action_permissions import ActionPermission, ActionObjectPermission

from syft.service.job.job_stash import Job
from syft.client.syncing import resolve_hierarchical

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /Users/eelco/dev/PySyft


In [2]:
node_low = sy.orchestra.launch(
    name="test_l",
    node_side_type="low",
    dev_mode=True,
    reset=True,
    local_db=True,
    n_consumers=1,
    create_producer=True,
)
node_high = sy.orchestra.launch(
    name="test_h",
    node_side_type="high",
    dev_mode=True,
    reset=True,
    local_db=True,
    n_consumers=1,
    create_producer=True,
)

Staging Protocol Changes...
SQLite Store Path:
!open file:///var/folders/pn/f6xkq7mx683g5jkyt91gqyzw0000gn/T/8a1c04544655402190588aec30079bc3.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=6947226323324fd7a8f80193f50a9d3a in_memory=True
Created default worker pool.
Data Migrated to latest version !!!
Staging Protocol Changes...
SQLite Store Path:
!open file:///var/folders/pn/f6xkq7mx683g5jkyt91gqyzw0000gn/T/8212e6797fde4c3fba4fc53ab555a886.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=e00258044b3f4bce8c9526e56f25aa81 in_memory=True
Created default worker pool.
Data Migrated to latest version !!!


In [3]:
client_low = node_low.login(email="info@openmined.org", password="changethis")
client_high = node_high.login(email="info@openmined.org", password="changethis")

Logged into <test_l: Low side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

Logged into <test_h: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

In [4]:
client_low.register(email="newuser@openmined.org", name="John Doe", password="pw", password_verify="pw")
client_low_ds = node_low.login(email="newuser@openmined.org", password="pw")

Logged into <test_l: Low side Domain> as <newuser@openmined.org>


# create datasets

In [5]:
# third party
import numpy as np

In [6]:
mock_high = np.array([10, 11, 12, 13, 14])
private_high = np.array([15, 16, 17, 18, 19])

dataset_high = sy.Dataset(
    name="my-dataset",
    description="abc",
    asset_list=[
        sy.Asset(
            name="numpy-data",
            mock=mock_high,
            data=private_high,
            shape=private_high.shape,
            mock_is_real=True,
        )
    ],
)

client_high.upload_dataset(dataset_high)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 11.38it/s]

Uploading: numpy-data


SyftSuccess: Dataset uploaded to 'test_h'. To see the datasets uploaded by a client on this node, use command `[your_client].datasets`

In [7]:
mock_low = np.array([0, 1, 2, 3, 4])  # do_high.mock
# private_low = np.array([5, 6, 7, 8, 9])  # AOEmpty? create new type AO

dataset_low = sy.Dataset(
    id=dataset_high.id,
    name="my-dataset",
    description="abc",
    asset_list=[
        sy.Asset(
            name="numpy-data",
            mock=mock_low,
            data=ActionObject.empty(data_node_id=client_high.id),
            shape=mock_low.shape,
            mock_is_real=True,
        )
    ],
)

res = client_low.upload_dataset(dataset_low)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 26.98it/s]

Uploading: numpy-data


In [8]:
data_low = client_low_ds.datasets[0].assets[0]


@sy.syft_function_single_use(data=data_low)
def compute_mean(data) -> float:
    print("computing mean, please stand by...")
    return data.mean()


compute_mean(data=data_low.mock)

SyftSuccess: Syft function 'compute_mean' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

SyftInfo: Creating a node with n_consumers=2 (the default value)
Staging Protocol Changes...
SQLite Store Path:
!open file:///var/folders/pn/f6xkq7mx683g5jkyt91gqyzw0000gn/T/ef45224b11764973a7da2452d798467c.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=f4e64a43645942c2bc7704323e124416 in_memory=True
Created default worker pool.
Data Migrated to latest version !!!
Logged into <ephemeral_node_compute_mean_8511: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

Approving request for domain ephemeral_node_compute_mean_8511
override True
computing mean, please stand by...
SyftInfo: Landing the ephmeral node...


```python
Pointer
```
2.0

SyftInfo: Node Landed!


In [9]:
client_low_ds.code.request_code_execution(compute_mean)

syft.service.request.request.Request

In [10]:
request = client_low.requests[0]

In [11]:
code = client_low.code[0]

In [12]:
from syft.client.syncing import compare_states

low_state = client_low.sync.get_state()
high_state = client_high.sync.get_state()

diff_state = compare_states(low_state, high_state)

In [13]:
diff_state.diffs

[syft.service.sync.diff_state.Diff, syft.service.sync.diff_state.Diff]

In [15]:
# state.objs_to_sync
from syft.client.syncing import resolve, resolve_hierarchical
low_items_to_sync, high_items_to_sync = resolve_hierarchical(diff_state)

┏━ Request(f8cfa9ad472447c08b860ace443dc057) - State: NEW ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                                                                                                 ┃
┃  ╭─ Low side ────────────────────────────────────────────────────────────────────────────────────────────────╮  ┃
┃  │                                                                                                           │  ┃
┃  │  class Request:                                                                                           │  ┃
┃  │  ‎ ‎ ‎ ‎ request_time = 2024-02-21 14:48:59                                                                   │  ┃
┃  │  ‎ ‎ ‎ ‎ updated_at = None                                                                                    │  ┃
┃  │  ‎ ‎ ‎ ‎ status = RequestStatus.PENDING                                                                       │  ┃
┃  │  ‎ ‎ ‎ ‎ changes = [                                                                                          │  ┃
┃  │  ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ UserCodeStatusChange                                                                             │  ┃
┃  │  ‎ ‎ ‎ ‎ ]                                                                                                    │  ┃
┃  │  ‎ ‎ ‎ ‎ requesting_user_verify_key = b6b3630c2b3c751e1b23dc0b9840671e84aa957066707391960a04c6c780678c        │  ┃
┃  │                                                                                                           │  ┃
┃  ╰───────────────────────────────────────────────────────────────────────────────────────────────────────────╯  ┃
┃  ╭─ High side ───────────────────────────────────────────────────────────────────────────────────────────────╮  ┃
┃  │ None                                                                                                      │  ┃
┃  ╰───────────────────────────────────────────────────────────────────────────────────────────────────────────╯  ┃
┃                                                                                                                 ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

┏━ UserCode(4c83ba20bca44285b42d8f4aae32b511) - State: NEW ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
     ┃                                                                                                            ┃
     ┃  ╭─ Low side ───────────────────────────────────────────────────────────────────────────────────────────╮  ┃
     ┃  │                                                                                                      │  ┃
     ┃  │  class UserCode:                                                                                     │  ┃
     ┃  │  ‎ ‎ ‎ ‎ service_func_name = compute_mean                                                                │  ┃
     ┃  │  ‎ ‎ ‎ ‎ input_owners = [                                                                                │  ┃
     ┃  │  ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ test_l                                                                                      │  ┃
     ┃  │  ‎ ‎ ‎ ‎ ]                                                                                               │  ┃
     ┃  │  ‎ ‎ ‎ ‎ code_status = [                                                                                 │  ┃
     ┃  │  ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ Node: test_l, Status: pending                                                               │  ┃
     ┃  │  ‎ ‎ ‎ ‎ ]                                                                                               │  ┃
     ┃  │  ‎ ‎ ‎ ‎ worker_pool_name = default-pool                                                                 │  ┃
     ┃  │                                                                                                      │  ┃
     ┃  ╰──────────────────────────────────────────────────────────────────────────────────────────────────────╯  ┃
     ┃  ╭─ High side ──────────────────────────────────────────────────────────────────────────────────────────╮  ┃
     ┃  │ None                                                                                                 │  ┃
     ┃  ╰──────────────────────────────────────────────────────────────────────────────────────────────────────╯  ┃
     ┃                                                                                                            ┃
     ┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

Do you want to keep the low state or the high state for these objects? choose 'low' or 'high'


 low


In [16]:
print("Low items to sync")
print(low_items_to_sync)
print()
print("High items to sync")
print(high_items_to_sync)

Low items to sync
ResolveState(
  create_objs=[],
  update_objs=[],
  delete_objs=[]
)

High items to sync
ResolveState(
  create_objs=[syft.service.request.request.Request, syft.service.code.user_code.UserCode],
  update_objs=[],
  delete_objs=[]
)


- What do we do with properties here?
  - just set them to None

In [17]:
def get_stash_for_item(item, node):
    services = list(node.python_node.service_path_map.values())

    all_stashes = dict()
    for serv in services:
        if (_stash := getattr(serv, "stash", None)) is not None:
            all_stashes[_stash.object_type] = _stash

    stash = all_stashes.get(type(item), None)
    return stash

In [18]:
def set_obj_ids(x, node_uid, verify_key):
    if hasattr(x, "__dict__") and isinstance(x, SyftObject):
        for prop, val in x.__dict__.items():
            if isinstance(val, (list, tuple)):
                for v in val:
                    set_obj_ids(v, node_uid, verify_key)
            elif isinstance(val, dict):
                for v in val.values():
                    set_obj_ids(v, node_uid, verify_key)
            else:
                set_obj_ids(val, node_uid, verify_key)
        x.syft_node_location=node_uid
        x.syft_client_verify_key=verify_key
        if hasattr(x, "node_uid"):
            x.node_uid=node_uid
                    
                    
    # TODO: hadnle nested collections
            

In [19]:
def add_permissions_for_actionobject(obj, node_to, node_from):
    _id = obj.id.id 
    blob_id = obj.syft_blob_storage_entry_id
    all_permissions = node_from.python_node.get_service("actionservice").store.permissions[_id]
    read_permissions = [x for x in all_permissions if "READ" in x]

    store_to = node_to.python_node.get_service("actionservice").store
    store_to_blob =node_to.python_node.get_service("blobstorageservice").stash.partition
    print("adding permissions")

    for read_permission in read_permissions:
        
        creds, perm_str = read_permission.split("_") 
        perm = ActionPermission[perm_str]
        permission = ActionObjectPermission(uid=_id, permission=perm, credentials=SyftVerifyKey(creds))
        res = store_to.add_permission(permission)
        
        permission_blob = ActionObjectPermission(uid=blob_id, permission=perm, credentials=SyftVerifyKey(creds))        
        res_blob = store_to_blob.add_permission(permission_blob)

        print("Created permission", permission)
        print("Created blob permission", permission, "with blob id", blob_id)
        
    

In [20]:
def create_actionobject(action_object, client, node_to, node_from):
    print("syncing obj with blob id", action_object.syft_blob_storage_entry_id)
    action_object = action_object.refresh_object()
    action_object.send(client)
#     res = client.api.services.action.set(action_object)
    print("Created", res, "blob_id:", action_object.syft_blob_storage_entry_id)

    if node_to.python_node.node_side_type.value == "low":
        add_permissions_for_actionobject(action_object, node_to, node_from)

In [21]:
from syft.client.api import NodeIdentity
from syft.service.code.user_code import UserCode

In [22]:
def transform_item(item, node):
    identity = NodeIdentity.from_node(node.python_node)
    if isinstance(item, UserCode):
        res = {}
        for key, val in item.status.status_dict.items():
            # todo, check if they are actually only two nodes
            res[identity] = code.status.status_dict[key]
        item.status.status_dict=res
    return item

In [23]:
def add_permissions_for_item(item, node_to, node_from):
    _id = item.id 
    all_permissions = node_from.python_node.get_service("jobservice").stash.partition.permissions[_id]
    read_permissions = [x for x in all_permissions if "READ" in x]

    store_to = node_to.python_node.get_service("jobservice").stash.partition
    for read_permission in read_permissions:
        creds, perm_str = read_permission.split("_") 
        perm = ActionPermission[perm_str]
        permission = ActionObjectPermission(uid=_id, permission=perm, credentials=SyftVerifyKey(creds))
        res = store_to.add_permission(permission)
        print("Created permission", res)

In [24]:
from syft.service.sync.diff_state import ResolveState

def sync_items(state: ResolveState, node_to, node_from, client):
    if len(state.delete_objs):
        raise NotImplementedError("TODO delete objects")
    
    items = state.create_objs + state.update_objs
    for item in items:
        
        if isinstance(item, ActionObject):
            create_actionobject(item, client, node_to, node_from)

        else:
                
            item = transform_item(item, node_to)
            stash = get_stash_for_item(item, node_to)
            creds=client.verify_key

            set_obj_ids(item, client.id, creds)
            exists = stash.get_by_uid(creds, item.id).ok() is not None
            if exists:
                res = stash.update(creds, item)
            else:
#                 res = stash.delete_by_uid(node.python_node.verify_key, item.id)
                res = stash.set(creds, item)
            if not res.is_ok():
                raise ValueError("")
            else:
                print("created", item)
                if isinstance(item, Job) and node_to.python_node.node_side_type.value == "low":
                    add_permissions_for_item(item, node_to, node_from)
    client._fetch_api(client.credentials)

In [25]:
sync_items(low_items_to_sync, node_low, node_high, client_low)

In [26]:
sync_items(high_items_to_sync, node_high, node_low, client_high)

created syft.service.request.request.Request
created syft.service.code.user_code.UserCode


In [27]:
# Verify state is the same

from syft.client.syncing import compare_states

low_state = client_low.sync.get_state()
high_state = client_high.sync.get_state()

diff_state = compare_states(low_state, high_state)

diff_state.diffs

[syft.service.sync.diff_state.Diff, syft.service.sync.diff_state.Diff]

# Run code high and sync back result

In [28]:
data_high = client_high.datasets[0].assets[0]

In [29]:
job_high = client_high.code.compute_mean(data=data_high, blocking=False)
display(job_high)

```python
class Job:
    id: UID = 71fdf2da7d0347a19591a9fd16eebab3
    status: created
    has_parent: False
    result: syft.service.action.action_data_empty.ObjectNotReady
    logs:

0 
    
```

In [30]:
# wait for the result
job_high.wait().get()

override True


21/02/24 16:50:46 FUNCTION LOG (71fdf2da7d0347a19591a9fd16eebab3): computing mean, please stand by...


17.0

In [31]:
job_info = job_high.info(public_metadata=True, result=True)

In [32]:
job_info.result

```python
Pointer
```
17.0

In [33]:
request = client_high.requests[0]
result_obj = job_high.result

print("result obj", result_obj.id)
accept_res = request.accept_by_depositing_result(job_info)

# request.approve()
# TODO: FIX THIS
request = client_high.requests[0]
code = request.code
log = job_high._get_log_objs()

result obj 983cae4ce1de47f79b7dbd47b3ccb2e2
Approving request for domain test_h
Approving request for domain test_h
ADDING PERMISSION [READ: 983cae4ce1de47f79b7dbd47b3ccb2e2 as b6b3630c2b3c751e1b23dc0b9840671e84aa957066707391960a04c6c780678c] 983cae4ce1de47f79b7dbd47b3ccb2e2
returning existing job
setting permission
None
None


In [34]:
action_store_high = node_high.python_node.get_service("actionservice").store
blob_store_high = node_high.python_node.get_service("blobstorageservice").stash.partition
assert f"{client_low_ds.verify_key}_READ" in action_store_high.permissions[job_high.result.id.id]
assert f"{client_low_ds.verify_key}_READ" in blob_store_high.permissions[job_high.result.syft_blob_storage_entry_id]

## Sync back to low

In [35]:
low_state = client_low.sync.get_state()
high_state = client_high.sync.get_state()

diff_state_2 = compare_states(low_state, high_state)

diff_state_2.diffs

[syft.service.sync.diff_state.Diff,
 syft.service.sync.diff_state.Diff,
 syft.service.sync.diff_state.Diff,
 syft.service.sync.diff_state.Diff,
 syft.service.sync.diff_state.Diff]

In [36]:
low_items_to_sync, high_items_to_sync = resolve_hierarchical(diff_state_2)

┏━ Request(f8cfa9ad472447c08b860ace443dc057) - State: DIFF ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                                                                                                 ┃
┃  ╭─ Low side ────────────────────────────────────────────────────────────────────────────────────────────────╮  ┃
┃  │                                                                                                           │  ┃
┃  │  class Request:                                                                                           │  ┃
┃  │  ‎ ‎ ‎ ‎ updated_at=None,                                                                                     │  ┃
┃  │  ‎ ‎ ‎ ‎ change=[                                                                                             │  ┃
┃  │  ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ UserCodeStatusChange                                                                             │  ┃
┃  │  ‎ ‎ ‎ ‎ ]                                                                                                    │  ┃
┃  │                                                                                                           │  ┃
┃  ╰───────────────────────────────────────────────────────────────────────────────────────────────────────────╯  ┃
┃  ╭─ High side ───────────────────────────────────────────────────────────────────────────────────────────────╮  ┃
┃  │                                                                                                           │  ┃
┃  │  class Request:                                                                                           │  ┃
┃  │  ‎ ‎ ‎ ‎ updated_at=2024-02-21 14:50:55,                                                                      │  ┃
┃  │  ‎ ‎ ‎ ‎ change=[                                                                                             │  ┃
┃  │  ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ UserCodeStatusChange                                                                             │  ┃
┃  │  ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ syft.service.request.request.ObjectMutation                                                      │  ┃
┃  │  ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ syft.service.request.request.ActionStoreChange                                                   │  ┃
┃  │  ‎ ‎ ‎ ‎ ]                                                                                                    │  ┃
┃  │                                                                                                           │  ┃
┃  ╰───────────────────────────────────────────────────────────────────────────────────────────────────────────╯  ┃
┃                                                                                                                 ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

┏━ UserCode(4c83ba20bca44285b42d8f4aae32b511) - State: DIFF ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
     ┃                                                                                                            ┃
     ┃  ╭─ Low side ───────────────────────────────────────────────────────────────────────────────────────────╮  ┃
     ┃  │                                                                                                      │  ┃
     ┃  │  class UserCode:                                                                                     │  ┃
     ┃  │  ‎ ‎ ‎ ‎ output_policy_state=b'...',                                                                     │  ┃
     ┃  │  ‎ ‎ ‎ ‎ status=(<UserCodeStatus.PENDING: 'pending'>, '')                                                │  ┃
     ┃  │                                                                                                      │  ┃
     ┃  ╰──────────────────────────────────────────────────────────────────────────────────────────────────────╯  ┃
     ┃  ╭─ High side ──────────────────────────────────────────────────────────────────────────────────────────╮  ┃
     ┃  │                                                                                                      │  ┃
     ┃  │  class UserCode:                                                                                     │  ┃
     ┃  │  ‎ ‎ ‎ ‎ output_policy_state=b'\x00\x00\x00\x00\xb9\x01\x00\x00\x00\x00...',                             │  ┃
     ┃  │  ‎ ‎ ‎ ‎ status=(<UserCodeStatus.APPROVED: 'approved'>, '')                                              │  ┃
     ┃  │                                                                                                      │  ┃
     ┃  ╰──────────────────────────────────────────────────────────────────────────────────────────────────────╯  ┃
     ┃                                                                                                            ┃
     ┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

┏━ Job(71fdf2da7d0347a19591a9fd16eebab3) - State: NEW ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
          ┃                                                                                                       ┃
          ┃  ╭─ Low side ──────────────────────────────────────────────────────────────────────────────────────╮  ┃
          ┃  │ None                                                                                            │  ┃
          ┃  ╰─────────────────────────────────────────────────────────────────────────────────────────────────╯  ┃
          ┃  ╭─ High side ─────────────────────────────────────────────────────────────────────────────────────╮  ┃
          ┃  │                                                                                                 │  ┃
          ┃  │  class Job:                                                                                     │  ┃
          ┃  │  ‎ ‎ ‎ ‎ id = 71fdf2da7d0347a19591a9fd16eebab3                                                      │  ┃
          ┃  │  ‎ ‎ ‎ ‎ result = 17.0                                                                              │  ┃
          ┃  │  ‎ ‎ ‎ ‎ resolved = True                                                                            │  ┃
          ┃  │  ‎ ‎ ‎ ‎ progress =                                                                                 │  ┃
          ┃  │  ‎ ‎ ‎ ‎ creation_time = 2024-02-21 16:50:44.515371                                                 │  ┃
          ┃  │                                                                                                 │  ┃
          ┃  ╰─────────────────────────────────────────────────────────────────────────────────────────────────╯  ┃
          ┃                                                                                                       ┃
          ┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

┏━ SyftLog(dcd0072f437348c3ae62fff9d4110e5c) - State: NEW ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
               ┃                                                                                                  ┃
               ┃  ╭─ Low side ─────────────────────────────────────────────────────────────────────────────────╮  ┃
               ┃  │ None                                                                                       │  ┃
               ┃  ╰────────────────────────────────────────────────────────────────────────────────────────────╯  ┃
               ┃  ╭─ High side ────────────────────────────────────────────────────────────────────────────────╮  ┃
               ┃  │                                                                                            │  ┃
               ┃  │  class SyftLog:                                                                            │  ┃
               ┃  │  ‎ ‎ ‎ ‎ stdout = computing mean, please stand by...                                           │  ┃
               ┃  │                                                                                            │  ┃
               ┃  │  ‎ ‎ ‎ ‎ stderr =                                                                              │  ┃
               ┃  │                                                                                            │  ┃
               ┃  ╰────────────────────────────────────────────────────────────────────────────────────────────╯  ┃
               ┃                                                                                                  ┃
               ┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

┏━ ActionObject(983cae4ce1de47f79b7dbd47b3ccb2e2) - State: NEW ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
               ┃                                                                                                  ┃
               ┃  ╭─ Low side ─────────────────────────────────────────────────────────────────────────────────╮  ┃
               ┃  │ None                                                                                       │  ┃
               ┃  ╰────────────────────────────────────────────────────────────────────────────────────────────╯  ┃
               ┃  ╭─ High side ────────────────────────────────────────────────────────────────────────────────╮  ┃
               ┃  │                                                                                            │  ┃
               ┃  │  Pointer:                                                                                  │  ┃
               ┃  │  17.0                                                                                      │  ┃
               ┃  │                                                                                            │  ┃
               ┃  ╰────────────────────────────────────────────────────────────────────────────────────────────╯  ┃
               ┃                                                                                                  ┃
               ┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

Do you want to keep the low state or the high state for these objects? choose 'low' or 'high'


 high


In [37]:
print("Low items to sync")
print(low_items_to_sync)
print()
print("High items to sync")
print(high_items_to_sync)

Low items to sync
ResolveState(
  create_objs=[syft.service.job.job_stash.Job, syft.service.log.log.SyftLog, Pointer:
17.0],
  update_objs=[syft.service.request.request.Request, syft.service.code.user_code.UserCode],
  delete_objs=[]
)

High items to sync
ResolveState(
  create_objs=[],
  update_objs=[],
  delete_objs=[]
)


In [38]:
sync_items(low_items_to_sync, node_low, node_high, client_low)

created syft.service.job.job_stash.Job
Created permission None
Created permission None
created syft.service.log.log.SyftLog
syncing obj with blob id 43d6fd556f9a436b82893e6b5906ad5b
Created SyftSuccess: Dataset uploaded to 'test_l'. To see the datasets uploaded by a client on this node, use command `[your_client].datasets` blob_id: 43d6fd556f9a436b82893e6b5906ad5b
adding permissions
Created permission [READ: 983cae4ce1de47f79b7dbd47b3ccb2e2 as b6b3630c2b3c751e1b23dc0b9840671e84aa957066707391960a04c6c780678c]
Created blob permission [READ: 983cae4ce1de47f79b7dbd47b3ccb2e2 as b6b3630c2b3c751e1b23dc0b9840671e84aa957066707391960a04c6c780678c] with blob id 43d6fd556f9a436b82893e6b5906ad5b
Created permission [READ: 983cae4ce1de47f79b7dbd47b3ccb2e2 as d311a667006cbb56614a062b2bbc7a733b5a4a8edd5293e0f34bb5c75f51277d]
Created blob permission [READ: 983cae4ce1de47f79b7dbd47b3ccb2e2 as d311a667006cbb56614a062b2bbc7a733b5a4a8edd5293e0f34bb5c75f51277d] with blob id 43d6fd556f9a436b82893e6b5906ad5b


In [39]:
sync_items(high_items_to_sync, node_high, node_low, client_high)

In [40]:
action_store_low = node_low.python_node.get_service("actionservice").store
blob_store_low = node_low.python_node.get_service("blobstorageservice").stash.partition
assert f"{client_low_ds.verify_key}_READ" in action_store_low.permissions[job_high.result.id.id]
assert f"{client_low_ds.verify_key}_READ" in blob_store_low.permissions[job_high.result.syft_blob_storage_entry_id]

# Run code low

## Run

In [41]:
res_low = client_low_ds.code.compute_mean(data=data_low)

override False


In [42]:
res_low.get()

17.0

In [43]:
assert res_low.get() == private_high.mean()
assert res_low.id == job_high.result.id.id == code.output_policy.last_output_ids[0].id.id
assert job_high.result.syft_blob_storage_entry_id == res_low.syft_blob_storage_entry_id

In [44]:
private_high.mean()

17.0

In [45]:
job_low = client_low_ds.code.compute_mean(data=data_low, blocking=False)

4c83ba20bca44285b42d8f4aae32b511


SyftWarning: There are existing jobs for this user code, returning the latest one

In [46]:
job_low.wait().get()

17.0

In [47]:
assert job_low.id == job_high.id
assert job_low.result.id == job_high.result.id
assert job_low.result.syft_blob_storage_entry_id == job_high.result.syft_blob_storage_entry_id